In [9]:
# -------------------------------
# Imports nécessaires
# -------------------------------
import psycopg2
import requests
import json

# -------------------------------
# Connexion PostgreSQL directe
# -------------------------------
PG_HOST = "localhost"
PG_PORT = 5432
PG_DATABASE = "rag_chatbot"
PG_USER = "postgres"
PG_PASSWORD = "nour123"

def connect_pg():
    """Connecte à PostgreSQL"""
    return psycopg2.connect(
        host=PG_HOST,
        port=PG_PORT,
        database=PG_DATABASE,
        user=PG_USER,
        password=PG_PASSWORD
    )

# -------------------------------
# Nettoyer le corpus
# -------------------------------
def load_corpus(path):
    """Charge le fichier et nettoie les lignes"""
    corpus = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith("<"):
                # enlever les préfixes type "h:" ou "c:"
                line = line.replace("h:", "").replace("c:", "").strip()
                corpus.append(line)
    return corpus

# -------------------------------
# Générer embeddings via Ollama
# -------------------------------
def embed(text):
    """Génère embedding via Ollama Llama3"""
    r = requests.post(
        "http://localhost:11434/api/embeddings",
        json={"model": "llama3", "prompt": text}
    )
    return r.json()["embedding"]

# -------------------------------
# Créer la table embeddings
# -------------------------------
def create_table():
    conn = connect_pg()
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS embeddings;")
    cur.execute("""
        CREATE TABLE embeddings (
            id SERIAL PRIMARY KEY,
            corpus TEXT,
            embedding VECTOR(4096)
        );
    """)
    conn.commit()
    conn.close()
    print("Table embeddings créée avec succès !")

# -------------------------------
# Insérer les embeddings
# -------------------------------
def insert_embeddings(corpus_list):
    conn = connect_pg()
    cur = conn.cursor()
    for text in corpus_list:
        emb = embed(text)
        # Convertir en string JSON pour PostgreSQL VECTOR
        emb_str = "[" + ",".join(map(str, emb)) + "]"
        cur.execute(
            "INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s)",
            (text, emb_str)
        )
    conn.commit()
    conn.close()
    print(f"{len(corpus_list)} embeddings insérés dans la base.")

# -------------------------------
# Recherche de textes similaires
# -------------------------------
def search_embeddings(query, top_k=3):
    q_emb = embed(query)
    q_emb_str = "[" + ",".join(map(str, q_emb)) + "]"
    conn = connect_pg()
    cur = conn.cursor()
    cur.execute("""
        SELECT corpus, 1 - (embedding <=> %s::vector) AS score
        FROM embeddings
        ORDER BY embedding <=> %s::vector
        LIMIT %s;
    """, (q_emb_str, q_emb_str, top_k))
    res = cur.fetchall()
    conn.close()
    return res

# -------------------------------
# -------------------------------
# EXEMPLE D'UTILISATION
# -------------------------------

# 1️⃣ Chemin vers le fichier corpus
corpus_file = r"D:\ENIS\3éme\AI_Fadoua\chatBot\Chatbot-RAG\data\fch.txt"

# 2️⃣ Charger et nettoyer le corpus
corpus_list = load_corpus(corpus_file)

# 3️⃣ Créer la table
create_table()

# 4️⃣ Insérer les embeddings
insert_embeddings(corpus_list)

# 5️⃣ Tester la recherche
query = "Bonjour!"
results = search_embeddings(query)
print("Résultats les plus proches :")
for r in results:
    print(r)


Table embeddings créée avec succès !
139 embeddings insérés dans la base.
Résultats les plus proches :
('oui [rire]', 0.6347352259101549)
('oui oui oui oui oui', 0.5424376705493125)
('ah oui', 0.5419235098278321)
